In [3]:

import json
from pathlib import Path
from tqdm import tqdm
from openai import OpenAI
import time
from datetime import datetime
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI()

# Get the current date
current_date = datetime.now().strftime("%b %d, %Y")

# Set up directories
unlabeled_dir = Path('flyer_data')
labeled_dir = Path('labeled_flyer_data')
labeled_dir.mkdir(exist_ok=True)

def log_info(message):
    """Log information to a file."""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open('batch_processing_log.txt', 'a') as log_file:
        log_file.write(f"{timestamp} - {message}\n")
    print(message)


In [28]:

system_prompt = """
Your goal is to help shoppers find the best deals. You will be given an image of a product ad and its price.
Calculate the price per unit and recommend what the user should do with the item. If at all possible,
get the unit as a weight or volume. Especially for food items, this is usually the best way to compare deals.
"""

def prepare_deal_input(item, flyer_filename):
    price = float(item['price'])
    return {
        "custom_id": f"{flyer_filename}|{item['id']}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o-mini-2024-07-18",
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": [
                    {"type": "text", "text": f"Today is {current_date}. Evaluate {item['name']} priced at ${price:.2f}."},
                    {"type": "image_url", "image_url": {"url": item['cutout_image_url']}}
                ]}
            ],
            "max_tokens": 100,
            "response_format": {
                "type": "json_schema",
                "json_schema": {
                    "name": "deal_schema",
                    "schema": {
                        "type": "object",
                        "properties": {
                            "price_per_unit": {
                                "description": "Formatted as $X.XX/unit, where unit is one of: lb, oz, g, kg, gal, qt, pt, fl oz, ea.",
                                "type": "string"
                            },
                            "user_recommendation": {
                                "description": "Tell the user what to do with this item so they can save money. For example, 'Buy bulk and fill your freezer!', 'Wait for a better deal', 'Buy if you need it soon', 'This deal is so good you should buy and resell it!'",
                                "type": "string"
                            },
                            "deal_score": {
                                "description": "A score from 1 to 5 indicating how good the deal is. 1 is worse than average, 2 is around the expected price, and 5 is that rare smoking unicorn deal.",
                                "type": "integer",
                                "minimum": 1,
                                "maximum": 5
                            },
                            "meal_prep_score": {
                                "description": "A score from 1 to 5 indicating how good the deal is for meal-prepping. 1 is anything that isn't a food item, 2 is food that isn't very nutritious (soft drinks, candy, chips), 3 is high quality sweets (pastries, ice cream, etc.), 4 is good canned or frozen food, and 5 is fresh produce, meat, dairy, or other high quality food.",
                                "type": "integer",
                                "minimum": 1,
                                "maximum": 5
                            },
                            "in_season": {
                                "description": "true if the item is in season, false otherwise.",
                                "type": "boolean"
                            }
                        },
                        "additionalProperties": False
                    }
                }
            }
        }
    }

In [29]:
# Cell 3: Load flyer data, prepare batch input, split into batches, and estimate cost

def estimate_tokens(text, has_image=True):
    """Estimate the number of tokens in the given text, plus image tokens if applicable."""
    text_tokens = len(text) // 4  # Using the heuristic that 1 token ~= 4 chars in English
    image_tokens = 8500 if has_image else 0  # Add 8500 tokens for each image
    return text_tokens, image_tokens

def estimate_cost(text_tokens, image_tokens, max_output_tokens=100):
    """Estimate the cost based on input tokens, image tokens, and output tokens."""
    image_cost = (image_tokens / 1_000_000) * 0.15
    input_text_cost = (text_tokens / 1_000_000) * 0.075
    output_cost = (max_output_tokens / 1_000_000) * 0.300
    return image_cost + input_text_cost + output_cost

batch_input = []
flyers_to_process = []
total_text_tokens = 0
total_image_tokens = 0
total_cost = 0
MAX_TOKENS_PER_BATCH = 1_800_000  # We'll keep this the same for now
batches = []
current_batch = []
current_batch_tokens = 0

for flyer_file in unlabeled_dir.glob('*.json'):
    labeled_file = labeled_dir / flyer_file.name
    if not labeled_file.exists():
        with open(flyer_file, 'r') as f:
            flyer_data = json.load(f)
        
        flyers_to_process.append(flyer_file.name)
        
        for item in flyer_data['items']:
            input_data = prepare_deal_input(item, flyer_file.name)
            item_text = json.dumps(input_data)
            text_tokens, image_tokens = estimate_tokens(item_text)
            item_tokens = text_tokens + image_tokens
            item_cost = estimate_cost(text_tokens, image_tokens)
            
            if current_batch_tokens + item_tokens > MAX_TOKENS_PER_BATCH:
                batches.append(current_batch)
                current_batch = []
                current_batch_tokens = 0
            
            current_batch.append(input_data)
            current_batch_tokens += item_tokens
            total_text_tokens += text_tokens
            total_image_tokens += image_tokens
            total_cost += item_cost

if current_batch:
    batches.append(current_batch)

if not flyers_to_process:
    log_info("No new flyers to process. Exiting.")
    exit()

total_tokens = total_text_tokens + total_image_tokens

log_info(f"Loaded {len(flyers_to_process)} unlabeled flyers")
log_info(f"Prepared {sum(len(batch) for batch in batches)} items for batch processing")
log_info(f"Estimated total tokens: {total_tokens}")
log_info(f"  - Text tokens: {total_text_tokens}")
log_info(f"  - Image tokens: {total_image_tokens}")
log_info(f"Number of batches: {len(batches)}")
log_info(f"Estimated total cost: ${total_cost:.2f}")

# Save flyers_to_process for later use
with open('flyers_to_process.json', 'w') as f:
    json.dump(flyers_to_process, f)

log_info("Flyers to process saved: flyers_to_process.json")

Loaded 63 unlabeled flyers
Prepared 4360 items for batch processing
Estimated total tokens: 39420013
  - Text tokens: 2360013
  - Image tokens: 37060000
Number of batches: 22
Estimated total cost: $5.87
Flyers to process saved: flyers_to_process.json


In [30]:
batch_ids = []

for i, batch in enumerate(batches):
    batch_filename = f'batch_input_{i}.jsonl'
    with open(batch_filename, 'w') as f:
        for item in batch:
            f.write(json.dumps(item) + '\n')
    
    batch_input_file = client.files.create(
        file=open(batch_filename, "rb"),
        purpose="batch"
    )

    log_info(f"Uploaded batch input file {i} with ID: {batch_input_file.id}")

    batch = client.batches.create(
        input_file_id=batch_input_file.id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={
            "description": f"Cheap deals analysis batch {i}"
        }
    )

    batch_ids.append(batch.id)
    log_info(f"Created batch {i} with ID: {batch.id}")

# Save batch IDs for later use
with open('batch_ids.json', 'w') as f:
    json.dump(batch_ids, f)

log_info(f"Created {len(batch_ids)} batches. Batch IDs saved to batch_ids.json")


Uploaded batch input file 0 with ID: file-v26wxjqgwUiHgWz3upPO0Hod
Created batch 0 with ID: batch_6719c9387b048190ac0cb5f6cb9da2e2
Uploaded batch input file 1 with ID: file-t4HAisEWSlCh5d9rswtJMY5q
Created batch 1 with ID: batch_6719c939b82c8190bc10920347fd3769
Uploaded batch input file 2 with ID: file-fh2BmLNHlgGm6f50Mxl2ovov
Created batch 2 with ID: batch_6719c93aa69481908c4c172954c2c1bb
Uploaded batch input file 3 with ID: file-7hrclOIFEMinZtcTx1kYMBKd
Created batch 3 with ID: batch_6719c93b8e34819089aa296ffa09d8b0
Uploaded batch input file 4 with ID: file-Ql7IMxE4EAiDcmR8DDDn6Ehm
Created batch 4 with ID: batch_6719c93c998c8190bfe6869bfbbe7953
Uploaded batch input file 5 with ID: file-5KRfIFUFz3ThN8iLB20QnzD1
Created batch 5 with ID: batch_6719c93da59081908e9d03fac0c26285
Uploaded batch input file 6 with ID: file-sC1xisaUdh6Fk8bZd1vQNV3p
Created batch 6 with ID: batch_6719c93e82d481909b481e88b10a93fe
Uploaded batch input file 7 with ID: file-pXSSHDHFLmO82YpnK3eQYRVu
Created batch 7 

KeyboardInterrupt: 

In [36]:
import time
from math import ceil

MAX_CONCURRENT_BATCHES = 5  # Adjust this based on the token limit and batch sizes
BATCH_CHECK_INTERVAL = 60  # Check batch status every 60 seconds

def process_batch_group(batch_ids, start_index, end_index):
    group_results = []
    active_batches = batch_ids[start_index:end_index]
    
    while active_batches:
        for batch_id in active_batches[:]:
            batch_status = client.batches.retrieve(batch_id)
            
            if batch_status.status == "completed":
                output_file_content = client.files.content(batch_status.output_file_id)
                output_text = output_file_content.read().decode('utf-8')
                batch_results = [json.loads(line) for line in output_text.strip().split('\n')]
                group_results.extend(batch_results)
                active_batches.remove(batch_id)
                log_info(f"Batch {batch_id} completed and processed.")
            elif batch_status.status == "failed":
                log_info(f"Batch {batch_id} failed. Error: {batch_status.errors}")
                active_batches.remove(batch_id)
        
        if active_batches:
            log_info(f"Waiting for {len(active_batches)} batches to complete...")
            time.sleep(BATCH_CHECK_INTERVAL)
    
    return group_results

# Load batch IDs and flyers to process
with open('batch_ids.json', 'r') as f:
    batch_ids = json.load(f)

with open('flyers_to_process.json', 'r') as f:
    flyers_to_process = json.load(f)

all_results = []
num_groups = ceil(len(batch_ids) / MAX_CONCURRENT_BATCHES)

for i in range(num_groups):
    start_index = i * MAX_CONCURRENT_BATCHES
    end_index = min((i + 1) * MAX_CONCURRENT_BATCHES, len(batch_ids))
    
    log_info(f"Processing batch group {i+1} of {num_groups}")
    group_results = process_batch_group(batch_ids, start_index, end_index)
    all_results.extend(group_results)
    
    log_info(f"Completed batch group {i+1}. Total results so far: {len(all_results)}")

# Group results by flyer
flyer_results = {}
for result in all_results:
    flyer_filename, item_id = result['custom_id'].split('|')
    if flyer_filename not in flyer_results:
        flyer_results[flyer_filename] = []
    flyer_results[flyer_filename].append((item_id, json.loads(result['response']['body'])))

# Update flyers with results and save labeled versions
for flyer_filename in flyers_to_process:
    with open(unlabeled_dir / flyer_filename, 'r') as f:
        flyer_data = json.load(f)
    
    for item in flyer_data['items']:
        item_results = next((r for r in flyer_results[flyer_filename] if r[0] == str(item['id'])), None)
        if item_results:
            item['deal_analysis'] = item_results[1]
    
    # Save labeled flyer
    labeled_file = labeled_dir / flyer_filename
    with open(labeled_file, 'w') as f:
        json.dump(flyer_data, f, indent=2)

log_info(f"Processed {len(all_results)} results and saved labeled flyers to {labeled_dir}")

# Clean up temporary files
os.remove('batch_ids.json')
os.remove('flyers_to_process.json')
for i in range(len(batch_ids)):
    os.remove(f'batch_input_{i}.jsonl')
log_info("Cleaned up temporary files")

# Display sample of labeled items
labeled_flyers = list(labeled_dir.glob('*.json'))
if labeled_flyers:
    print("\nSample of labeled items:")
    sample_flyer = labeled_flyers[0]
    print(f"Flyer: {sample_flyer.name}")
    with open(sample_flyer, 'r') as f:
        flyer_data = json.load(f)
    for item in flyer_data['items'][:5]:
        print(json.dumps(item, indent=2))
        print()
else:
    print("No labeled flyers found. Please check if the processing completed successfully.")

Processing batch group 1 of 4
Batch batch_6719c9387b048190ac0cb5f6cb9da2e2 completed and processed.
Batch batch_6719c939b82c8190bc10920347fd3769 completed and processed.
Batch batch_6719c93c998c8190bfe6869bfbbe7953 completed and processed.
Waiting for 2 batches to complete...


In [34]:
if os.path.exists('batch_ids.json'):
    with open('batch_ids.json', 'r') as f:
        batch_ids = json.load(f)
    
    for i, batch_id in enumerate(batch_ids):
        batch_status = client.batches.retrieve(batch_id)
        log_info(f"Batch {i} status: {batch_status.status}")
        log_info(f"Completed requests: {batch_status.request_counts.completed}")
        log_info(f"Failed requests: {batch_status.request_counts.failed}")
        
        if batch_status.status == "failed":
            log_info(f"Batch {i} failed. Retrieving error information...")
            
            if batch_status.error_file_id:
                error_file_content = client.files.content(batch_status.error_file_id)
                error_text = error_file_content.read().decode('utf-8')
                log_info("Error details:")
                print(error_text)
            else:
                log_info("No detailed error information available.")
            
            if batch_status.errors:
                log_info("Batch level errors:")
                for error in batch_status.errors:
                    if isinstance(error, tuple):
                        print(f"Error: {error}")
                    elif hasattr(error, 'message'):
                        print(f"Error: {error.message}")
                    else:
                        print(f"Error: {error}")
        print()  # Add a blank line between batch status reports
else:
    log_info("No active batches found. Please run the previous cells to create new batches.")

No active batches found. Please run the previous cells to create new batches.


In [4]:
# Cell to cancel the in-progress batch
import json
with open('batch_ids.json', 'r') as f:
    batch_ids = json.load(f)

for batch_id in batch_ids:
    try:
        client.batches.cancel(batch_id)
        log_info(f"Cancelled batch {batch_id}")
    except Exception as e:
        log_info(f"Error cancelling batch {batch_id}: {str(e)}")

# Clean up temporary files
os.remove('batch_ids.json')
os.remove('flyers_to_process.json')
for i in range(len(batch_ids)):
    os.remove(f'batch_input_{i}.jsonl')
log_info("Cleaned up temporary files")

Error cancelling batch batch_6719c9387b048190ac0cb5f6cb9da2e2: Error code: 409 - {'error': {'message': "Cannot cancel a batch with status 'completed'.", 'type': 'invalid_request_error', 'param': None, 'code': None}}
Error cancelling batch batch_6719c939b82c8190bc10920347fd3769: Error code: 409 - {'error': {'message': "Cannot cancel a batch with status 'completed'.", 'type': 'invalid_request_error', 'param': None, 'code': None}}
Error cancelling batch batch_6719c93aa69481908c4c172954c2c1bb: Error code: 409 - {'error': {'message': "Cannot cancel a batch with status 'completed'.", 'type': 'invalid_request_error', 'param': None, 'code': None}}
Cancelled batch batch_6719c93b8e34819089aa296ffa09d8b0
Error cancelling batch batch_6719c93c998c8190bfe6869bfbbe7953: Error code: 409 - {'error': {'message': "Cannot cancel a batch with status 'completed'.", 'type': 'invalid_request_error', 'param': None, 'code': None}}
Error cancelling batch batch_6719c93da59081908e9d03fac0c26285: Error code: 409 - 

In [ ]:
if not os.path.exists('batch_ids.json') or not os.path.exists('flyers_to_process.json'):
    log_info("No active batches or flyers to process found. Please run the previous cells to create new batches.")
    exit()

with open('batch_ids.json', 'r') as f:
    batch_ids = json.load(f)

with open('flyers_to_process.json', 'r') as f:
    flyers_to_process = json.load(f)

all_results = []

for batch_id in batch_ids:
    batch_status = client.batches.retrieve(batch_id)

    if batch_status.status == "completed":
        output_file_content = client.files.content(batch_status.output_file_id)
        output_text = output_file_content.read().decode('utf-8')
        
        # Process the output
        batch_results = [json.loads(line) for line in output_text.strip().split('\n')]
        all_results.extend(batch_results)
    else:
        log_info(f"Batch {batch_id} is not yet completed. Please check again later.")
        exit()

# Group results by flyer
flyer_results = {}
for result in all_results:
    flyer_filename, item_id = result['custom_id'].split('|')
    if flyer_filename not in flyer_results:
        flyer_results[flyer_filename] = []
    flyer_results[flyer_filename].append((item_id, json.loads(result['response']['body'])))

# Update flyers with results and save labeled versions
for flyer_filename in flyers_to_process:
    with open(unlabeled_dir / flyer_filename, 'r') as f:
        flyer_data = json.load(f)
    
    for item in flyer_data['items']:
        item_results = next((r for r in flyer_results[flyer_filename] if r[0] == str(item['id'])), None)
        if item_results:
            item['deal_analysis'] = item_results[1]
    
    # Save labeled flyer
    labeled_file = labeled_dir / flyer_filename
    with open(labeled_file, 'w') as f:
        json.dump(flyer_data, f, indent=2)

log_info(f"Processed {len(all_results)} results and saved labeled flyers to {labeled_dir}")

# Clean up temporary files
os.remove('batch_ids.json')
os.remove('flyers_to_process.json')
for i in range(len(batch_ids)):
    os.remove(f'batch_input_{i}.jsonl')
log_info("Cleaned up temporary files")

In [ ]:
labeled_flyers = list(labeled_dir.glob('*.json'))
if labeled_flyers:
    print("\nSample of labeled items:")
    sample_flyer = labeled_flyers[0]
    print(f"Flyer: {sample_flyer.name}")
    with open(sample_flyer, 'r') as f:
        flyer_data = json.load(f)
    for item in flyer_data['items'][:5]:
        print(json.dumps(item, indent=2))
        print()
else:
    print("No labeled flyers found. Please run the processing cell when the batch is completed.")

In [22]:
# Cell to retrieve and display results of completed batches

def get_completed_batch_results(batch_id):
    batch_status = client.batches.retrieve(batch_id)
    if batch_status.status == "completed":
        output_file_content = client.files.content(batch_status.output_file_id)
        output_text = output_file_content.read().decode('utf-8')
        return [json.loads(line) for line in output_text.strip().split('\n')]
    return []

# List of batch IDs that were completed
completed_batch_ids = [
    "batch_6719c33a60a481908a4b639aabcc9ea2",
    "batch_6719c33b97d08190baba5c2198e6d341",
    "batch_6719c33d6be881908e54e8f15f1d24d2",
    "batch_6719c34183d881909d84f283a4ea3e55",
    "batch_6719c348e3488190b99ee5a11a5452f0"
]

all_results = []

for batch_id in completed_batch_ids:
    results = get_completed_batch_results(batch_id)
    all_results.extend(results)
    log_info(f"Retrieved {len(results)} results from batch {batch_id}")

# Display a sample of the results
print("\nSample of completed results:")
for result in all_results[:5]:  # Display first 5 results
    print(f"\nCustom ID: {result['custom_id']}")
    response_body = result['response']['body']
    if isinstance(response_body, str):
        response_body = json.loads(response_body)
    print(f"Response: {json.dumps(response_body, indent=2)}")

# Save all results to a file
with open('completed_results.json', 'w') as f:
    json.dump(all_results, f, indent=2)

log_info(f"Total results retrieved: {len(all_results)}")
log_info("All results saved to completed_results.json")

Retrieved 199 results from batch batch_6719c33a60a481908a4b639aabcc9ea2
Retrieved 199 results from batch batch_6719c33b97d08190baba5c2198e6d341
Retrieved 199 results from batch batch_6719c33d6be881908e54e8f15f1d24d2
Retrieved 199 results from batch batch_6719c34183d881909d84f283a4ea3e55
Retrieved 198 results from batch batch_6719c348e3488190b99ee5a11a5452f0

Sample of completed results:

Custom ID: Big 5 Sporting Goods_Weekly Ad_20241021-20241023_6879293.json|884578110
Response: {
  "id": "chatcmpl-ALj6THNRN2SY7tNGUPM36mw62sicL",
  "object": "chat.completion",
  "created": 1729741629,
  "model": "gpt-4o-mini-2024-07-18",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\"price_per_unit\":\"$15.99/ea\",\"user_recommendation\":\"Buy if you need it soon\",\"deal_score\":3,\"meal_prep_score\":1,\"in_season\":false}",
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "

In [26]:
# Cell to process and sort food deals, including item name and Flipp URL

import json
from operator import itemgetter
from pathlib import Path

def parse_response(response):
    if isinstance(response['body'], str):
        body = json.loads(response['body'])
    else:
        body = response['body']
    
    if isinstance(body['choices'][0]['message']['content'], str):
        content = json.loads(body['choices'][0]['message']['content'])
    else:
        content = body['choices'][0]['message']['content']
    
    return content

def extract_ad_info(custom_id):
    parts = custom_id.split('|')
    flyer_filename = parts[0]
    item_id = parts[1]  # This is the individual advertisement ID
    flyer_parts = flyer_filename.split('_')
    store = flyer_parts[0]
    ad_type = flyer_parts[1]
    date_range = flyer_parts[2]
    flyer_id = flyer_parts[3].split('.')[0]
    return {
        'store': store,
        'ad_type': ad_type,
        'date_range': date_range,
        'flyer_id': flyer_id,
        'item_id': item_id,
        'flyer_filename': flyer_filename
    }

# Load the completed results
with open('completed_results.json', 'r') as f:
    all_results = json.load(f)

# Process and filter food deals
food_deals = []
unlabeled_dir = Path('flyer_data')

for result in all_results:
    content = parse_response(result['response'])
    
    # Check if it's a food item (meal_prep_score > 1)
    if content['meal_prep_score'] > 1:
        ad_info = extract_ad_info(result['custom_id'])
        
        # Load the original flyer data to get the item name
        with open(unlabeled_dir / ad_info['flyer_filename'], 'r') as f:
            flyer_data = json.load(f)
        
        # Find the item in the flyer data
        item = next((item for item in flyer_data['items'] if str(item['id']) == ad_info['item_id']), None)
        item_name = item['name'] if item else "Unknown Item"
        
        deal = {
            'store': ad_info['store'],
            'ad_type': ad_info['ad_type'],
            'date_range': ad_info['date_range'],
            'flyer_id': ad_info['flyer_id'],
            'item_id': ad_info['item_id'],
            'item_name': item_name,
            'flipp_url': f"https://flipp.com/en-us/item/{ad_info['item_id']}",
            'price_per_unit': content['price_per_unit'],
            'deal_score': content['deal_score'],
            'meal_prep_score': content['meal_prep_score'],
            'user_recommendation': content['user_recommendation'],
            'in_season': content['in_season']
        }
        food_deals.append(deal)

# Sort food deals by deal_score and meal_prep_score
sorted_food_deals = sorted(food_deals, key=itemgetter('deal_score', 'meal_prep_score'), reverse=True)

# Display top 10 food deals
print("Top 10 Food Deals:")
for i, deal in enumerate(sorted_food_deals[:10], 1):
    print(f"\n{i}. {deal['item_name']}")
    print(f"   Store: {deal['store']} - {deal['ad_type']} (Flyer ID: {deal['flyer_id']})")
    print(f"   Flipp URL: {deal['flipp_url']}")
    print(f"   Date Range: {deal['date_range']}")
    print(f"   Price: {deal['price_per_unit']}")
    print(f"   Deal Score: {deal['deal_score']}/5")
    print(f"   Meal Prep Score: {deal['meal_prep_score']}/5")
    print(f"   Recommendation: {deal['user_recommendation']}")
    print(f"   In Season: {'Yes' if deal['in_season'] else 'No'}")

# Save sorted food deals to a file
with open('sorted_food_deals.json', 'w') as f:
    json.dump(sorted_food_deals, f, indent=2)

print(f"\nTotal food deals found: {len(sorted_food_deals)}")
print("Sorted food deals saved to sorted_food_deals.json")

Top 10 Food Deals:

1. Turkey Breast
   Store: Smiths - Weekly Ad (Flyer ID: 6883659)
   Flipp URL: https://flipp.com/en-us/item/885001499
   Date Range: 20241023-20241029
   Price: $1.79/lb
   Deal Score: 5/5
   Meal Prep Score: 5/5
   Recommendation: Buy now and stock your freezer for later meals!
   In Season: Yes

2. Large Avocados
   Store: Smiths - Weekly Ad (Flyer ID: 6883659)
   Flipp URL: https://flipp.com/en-us/item/885001507
   Date Range: 20241023-20241029
   Price: $0.99/ea
   Deal Score: 5/5
   Meal Prep Score: 5/5
   Recommendation: Buy as many as you can use before they spoil!
   In Season: Yes

3. Boneless Pork Sirloin Roast
   Store: Smiths - Weekly Ad (Flyer ID: 6883659)
   Flipp URL: https://flipp.com/en-us/item/885001383
   Date Range: 20241023-20241029
   Price: $1.29/lb
   Deal Score: 5/5
   Meal Prep Score: 5/5
   Recommendation: This is a great price for pork. Buy and freeze for future meals!
   In Season: Yes

4. Campbell's Condensed Soup, 10.5-11.25 oz, StarK